Import library

In [1]:
import sqlite3 
import pandas as pd
import numpy as np
import matplotlib as plt

In [30]:
db_path = "/Users/shangweisong/Desktop/AIAP_Student_score/data/score.db"

conn = sqlite3.connect(db_path)
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)

print("Tables in the database:", tables)

# Load a specific table into a DataFrame

df = pd.read_sql_query("SELECT * FROM score;", conn)  # replace with your table name

# Preview the data
print(df.head())
conn.close

Tables in the database:     name
0  score
   index  number_of_siblings direct_admission     CCA learning_style  \
0      0                   0              Yes  Sports         Visual   
1      1                   2               No  Sports       Auditory   
2      2                   0              Yes    None         Visual   
3      3                   1               No   Clubs       Auditory   
4      4                   0               No  Sports       Auditory   

  student_id  gender tuition  final_test  n_male  n_female   age  \
0     ACN2BE  Female      No        69.0    14.0       2.0  16.0   
1     FGXIIZ  Female      No        47.0     4.0      19.0  16.0   
2     B9AI9F    Male      No        85.0    14.0       2.0  15.0   
3     FEVM1T  Female     Yes        64.0     2.0      20.0  15.0   
4     AXZN2E    Male      No        66.0    24.0       3.0  16.0   

   hours_per_week  attendance_rate sleep_time wake_time  mode_of_transport  \
0            10.0             91.0    

<function Connection.close()>